In [1]:
import cv2 as cv
import numpy as np
import PIL
from math import pi
import math
from matplotlib import pyplot as plt
import random
from tqdm import tqdm
from sklearn.cluster import DBSCAN
from scipy.spatial import ConvexHull, convex_hull_plot_2d
import skimage.io
from scipy import ndimage as ndi
from statistics import mean

print("Done")

Done


In [2]:
def do_image(original, thresh_lo, thresh_hi, erode_val, gauss_val, exponent, gauss_val2, R, G, B):
    # original = cv.cvtColor(cv.imread("download.png"), cv.COLOR_BGR2RGB)
    r_channel, g_channel, b_channel = cv.split(original)
    color_channels = [r_channel, g_channel, b_channel]
    alpha_channel = np.ones(b_channel.shape, dtype=b_channel.dtype) * 50 #creating a dummy alpha channel image.
    
    # fig, axs = plt.subplots(ncols = 3, figsize=(15, 15))
    
    border_int = 3
    img = cv.copyMakeBorder(cv.cvtColor(original, cv.COLOR_BGR2GRAY), border_int, border_int, border_int, border_int, cv.BORDER_CONSTANT, value=255)
    # axs[0].imshow(original)
    
    ret, thresh = cv.threshold(img, thresh_lo, thresh_hi, cv.THRESH_BINARY)
    dist = cv.distanceTransform(thresh, cv.DIST_L2, cv.DIST_MASK_PRECISE)
    dist = cv.GaussianBlur(dist, (gauss_val, gauss_val), 0)
    dist = dist[border_int:img.shape[0]-border_int, border_int:img.shape[1]-border_int]
    norm_dist = cv.normalize(dist, None, 0, 1.0, cv.NORM_MINMAX)
    # axs[1].imshow(norm_dist)
    alpha_channel = cv.bitwise_not((255 * norm_dist).astype(b_channel.dtype))
    
    kernel = np.ones((erode_val, erode_val), np.uint8)
    alpha_channel = cv.erode(alpha_channel, kernel)
    
    # axs[2].imshow(img_BGRA)
    
    alpha_mult_temp = np.zeros_like(color_channels, dtype=float)
    alpha_mult = np.zeros_like(alpha_channel, dtype=float)

    for row in range(np.shape(color_channels)[1]):
        for col in range(np.shape(color_channels)[2]):
            for chan in range(np.shape(color_channels)[0]):
                alpha_mult_temp[chan][row][col] = pow(abs(int(color_channels[chan][row][col])-default_vals[chan])/default_vals[chan], exponent) * 5
                if chan % 3 == 2:
                    if mean(alpha_mult_temp[..., row, col]) > 1:
                        alpha_channel[row][col] = alpha_channel[row][col] * 1.0
                    else:
                        alpha_channel[row][col] = alpha_channel[row][col] * mean(alpha_mult_temp[..., row, col])
    
    alpha_channel = cv.GaussianBlur(alpha_channel, (gauss_val2, gauss_val2), 0)
    img_BGRA = cv.merge((r_channel, g_channel, b_channel, alpha_channel))
    
    return img_BGRA, alpha_channel

def weight_func(markers, markers2):
    new_running_sum = 0
    for i, row in enumerate(markers):
        for j, cell in enumerate(row):
            if markers[i][j] == markers2[i][j]:
                new_running_sum = new_running_sum + 1
            else:
                new_running_sum = new_running_sum - 1
    return new_running_sum

print("Done")

Done


In [3]:
# parameters
thresh_lo = 157
thresh_hi = 200
erode_val = 10
gauss_val = 9
exponent = 0.18
gauss_val2 = 3
R = 135
G = 226
B = 192
starting_R = R
starting_G = G
starting_B = B
variability = 5

default_vals = [R, G, B]

img = cv.cvtColor(cv.imread("new_download2.png"), cv.COLOR_BGR2RGB)
img2 = cv.bitwise_not(cv.imread("new_download.png"))
starting_params = [thresh_lo, thresh_hi, erode_val, gauss_val, exponent, gauss_val2, R, G, B]

_, start_markers = do_image(img, *starting_params)
compare_img, start_markers2 = do_image(img2, *starting_params)
plt.imshow(start_markers2)
plt.show()
start_running_sum = weight_func(start_markers, start_markers2)
print("Starting sum: %d" % (start_running_sum))
best_params = starting_params
best_sum = start_running_sum

k = 0
iters = 50000
with tqdm(total=iters, unit="iteration", desc=str("Monte Carlo Param Search:")) as pbar:
    while k < iters:
        thresh_hi = random.randint(1,255) # default == 255
        thresh_lo = random.randint(0, thresh_hi) # default == 0
        erode_val = random.randint(0, 30) # default == 3
        gauss_val = random.choice([1, 3, 5, 7, 9])
        exponent = random.randint(10, 1000) / 100
        gauss_val2 = random.choice([1, 3, 5, 7, 9])
        R = random.randint(R-5, R+5)
        G = random.randint(G-5, G+5)
        B = random.randint(B-5, B+5)
        _, markers = do_image(img, thresh_lo, thresh_hi, erode_val, gauss_val, exponent, gauss_val2, R, G, B)
        
        new_running_sum = weight_func(markers, start_markers2)
        if new_running_sum > best_sum:
            best_sum = new_running_sum
            best_params = [thresh_lo, thresh_hi, erode_val, gauss_val, exponent, gauss_val2, R, G, B]
        k = k + 1
        pbar.set_postfix(Sum = new_running_sum, refresh=True)
        pbar.update(1)

original_img, markers = do_image(img, *best_params)
fig, axs = plt.subplots(ncols = 4, figsize=(15, 15))
axs[0].imshow(compare_img)
axs[1].imshow(start_markers2)
axs[2].imshow(original_img)
axs[3].imshow(markers)

new_running_sum = weight_func(markers, start_markers2)
print(new_running_sum)
print(*best_params)

error: OpenCV(4.8.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'
